In [1]:
## install
%pip install pandas
%pip install chardet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## import
import os
import pandas as pd
import chardet
import re
from bs4 import BeautifulSoup
from datetime import datetime

## Convert sources to csv

In [12]:
# 👹
import os
import pandas as pd
import csv

# Define folder path
folder_path = "/work/LauraSørineVoldgaard#8191/data/sources"

# Initialize an empty list to hold DataFrames for each file
regular_sources = []

# List of files to skip
skip_files = ["sources-21-08.txt", 
              "sources-22-10.txt", 
              "sources-22-05.txt", 
              "sources-22-09.txt", 
              "sources-22-06.txt", 
              "sources-22-03.txt",
              "sources-24-11.txt",
              "sources-23-03.txt"
             ]

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    # Check if the file should be skipped
    if file_name in skip_files:
        continue  # Skip this file entirely
    
    # Full file path
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the path is a file and ends with '.txt'
    if os.path.isfile(file_path) and file_name.endswith('.txt'):
        try:
            # Read the file into a dataframe
            sources = pd.read_csv(
                file_path, 
                delimiter='\t', 
                encoding='ISO-8859-1',
                quoting=csv.QUOTE_NONE,  # Handle quotes as literal characters
                on_bad_lines='skip',     # Skip malformed lines
                dtype=str,               # Read all columns as strings
                low_memory=False         # Avoid mixed-type warnings
            )
            
            # Define and assign column names
            column_names = ["textID", "words", "date", "country", "source", "url", "headline"]
            sources.columns = column_names
            
            # Append the dataframe to the list
            regular_sources.append(sources)
        except pd.errors.ParserError as e:
            print(f"ParserError in file: {file_name} - {e}")
            continue
        except Exception as e:
            print(f"General error in file: {file_name} - {e}")
            continue

# Concatenate all dataframes into one
if regular_sources:
    regular_sources = pd.concat(regular_sources, ignore_index=True)
    regular_sources
else:
    print("No valid data to display.")



In [13]:
regular_sources

,textID,words,date,country,source,url,headline
0,103118757,307,23-11-01,US,nytimes.com,https://www.mercurynews.com/2023/10/31/google-...,Google trims Bay Area jobs at offices it had l...
1,103118769,961,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/yac...,Yacolt teen works with Nike to make sneakers t...
2,103118770,422,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cla...,"Clark County housing market heats over summer,..."
3,103118772,399,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/oct/31/van...,Vancouver-based ZoomInfo reports revenue incre...
4,103118775,212,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cli...,Climate solutions topic of Nov. 15 TEDx event ...
...,...,...,...,...,...,...,...
11524825,43919073,369,22-01-31,??,audible.com,https://www.audible.com/search?keywords=lylah+...,Audiobooks matching keywords lylah james | <st...
11524826,43919075,1570,22-01-31,??,audible.com,https://www.audible.com/search?keywords=rober+...,Audiobooks matching keywords rober jordan | Au...
11524827,43919078,636,22-01-31,??,audible.com,https://www.audible.com/search?keywords=the+gi...,Audiobooks matching keywords the girls ive bee...
11524828,43919079,434,22-01-31,??,audible.com,https://www.audible.com/search?keywords=utopia...,Audiobooks matching keywords utopian fiction |...


In [14]:
# number of duplicate rows (after removing the NaN in body column rows)
duplicates = regular_sources[regular_sources['textID'].duplicated(keep=False)]
duplicates

,textID,words,date,country,source,url,headline
661621,Business | 16 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
662055,Cannock | 6 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
678318,Dudley | 6 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
972017,Business | 11 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
983837,Wolverhampton | 15 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1198291,PremiumPeter Rhodes | 16 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1939879,Toby Neal | 15 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1948850,Telford | 3 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1948852,Telford | 3 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1962425,PremiumPeter Rhodes | 15 hours ago,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# 👹 Load in all the susnomore CSV files

import os
import pandas as pd

# Define folder path
folder_path = "/work/LauraSørineVoldgaard#8191/data/sus_sources"

# Initialize an empty list to hold DataFrames for each file
susnomore_sources = []

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)  # Full file path
    
    # Check if the file is a CSV
    if os.path.isfile(file_path) and file_name.endswith('.csv'):
        try:
            # Read the CSV file into a DataFrame
            sources = pd.read_csv(file_path, dtype=str)  # Read all columns as strings
            
            # Append the DataFrame to the list
            susnomore_sources.append(sources)
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
            continue

# Concatenate all DataFrames into one
if susnomore_sources:
    susnomore_sources = pd.concat(susnomore_sources, ignore_index=True)
    susnomore_sources
else:
    print("No valid data to display.")


In [16]:
# check that all the months not included in regular sources is included in susnomore_sources
susnomore_sources

#susnomore_sources.sample(n=10)

#regular_sources[regular_sources['date'] == '23-04-01']

,textID,words,date,country,source,url,headline
0,33086945,735,21-08-01,US,forbes.com,https://www.forbes.com/sites/michaelalpiner/20...,Lackawanna Coal Mine Tour Offers Travelers A R...
1,33086946,1185,21-08-01,US,forbes.com,https://www.forbes.com/sites/splunk/2021/08/01...,Modernizing The Mission: How Data Innovation B...
2,33086959,227,21-08-01,US,npr.org,https://www.npr.org/programs/morning-edition/2...,"Morning Edition for July 29, 2021 : NPR"
3,33086961,506,21-08-01,US,nbcnews.com,https://www.nbcnews.com/politics/congress/sena...,Senate introduces the details of the bipartisa...
4,33086966,1034,21-08-01,US,theringer.com,https://www.theringer.com/nba/2021/7/28/225989...,Could Trading for Buddy Hield Put LeBron and t...
...,...,...,...,...,...,...,...
1004402,89566655,3298.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/11-men-share-e...,11 men share their experiences of abortion aft...
1004403,89566656,614.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/yes-ice-cream-...,"Yes, You Can Have Ice Cream Delivered Straight..."
1004404,89566657,287.0,22-06-30,US,Yahoo,https://www.yahoo.com/news/football-rumours-bo...,Borussia Dortmund set Jude Bellingham's price ...
1004405,89566570,384.0,22-06-30,US,The Hill,https://thehill.com/changing-america/enrichmen...,New Jersey is losing residents to states like ...


In [17]:
print(type(regular_sources))
print(type(susnomore_sources))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [18]:
import pandas as pd

# Concatenate the individual lists of DataFrames
#regular_sources_df = pd.concat(regular_sources, ignore_index=True)
#susnomore_sources_df = pd.concat(susnomore_sources, ignore_index=True)

# Concatenate the two resulting DataFrames
all_sources = pd.concat([regular_sources, susnomore_sources], ignore_index=True)

# Display the combined DataFrame
all_sources


,textID,words,date,country,source,url,headline
0,103118757,307,23-11-01,US,nytimes.com,https://www.mercurynews.com/2023/10/31/google-...,Google trims Bay Area jobs at offices it had l...
1,103118769,961,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/yac...,Yacolt teen works with Nike to make sneakers t...
2,103118770,422,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cla...,"Clark County housing market heats over summer,..."
3,103118772,399,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/oct/31/van...,Vancouver-based ZoomInfo reports revenue incre...
4,103118775,212,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cli...,Climate solutions topic of Nov. 15 TEDx event ...
...,...,...,...,...,...,...,...
12529232,89566655,3298.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/11-men-share-e...,11 men share their experiences of abortion aft...
12529233,89566656,614.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/yes-ice-cream-...,"Yes, You Can Have Ice Cream Delivered Straight..."
12529234,89566657,287.0,22-06-30,US,Yahoo,https://www.yahoo.com/news/football-rumours-bo...,Borussia Dortmund set Jude Bellingham's price ...
12529235,89566570,384.0,22-06-30,US,The Hill,https://thehill.com/changing-america/enrichmen...,New Jersey is losing residents to states like ...


In [10]:
print(type(all_sources))

<class 'pandas.core.frame.DataFrame'>


In [19]:
# number of duplicate rows (after removing the NaN in body column rows)
duplicates = all_sources[all_sources['textID'].duplicated(keep=False)]
duplicates

,textID,words,date,country,source,url,headline
661621,Business | 16 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
662055,Cannock | 6 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
678318,Dudley | 6 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
972017,Business | 11 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
983837,Wolverhampton | 15 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1198291,PremiumPeter Rhodes | 16 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1939879,Toby Neal | 15 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1948850,Telford | 3 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1948852,Telford | 3 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
1962425,PremiumPeter Rhodes | 15 hours ago,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Display rows with NaN values in any column
sources_rows_with_nan = all_sources[all_sources.isna().any(axis=1)]

# Print the rows with NaN values
sources_rows_with_nan

,textID,words,date,country,source,url,headline
661621,Business | 16 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
662049,Kidderminster | 4 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
662053,Wolverhampton | 4 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
662055,Cannock | 6 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
662077,Bridgnorth | 4 hours ago,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
12130731,116962521,451,24-11-01,US,NaN,https://www.broadwayworld.com/article/Photos-S...,AN EVENING WITH AARON LAZAR For Project ALS
12130732,116962522,258,24-11-01,US,NaN,https://www.broadwayworld.com/article/Photos-T...,TAMMY FAYE First Look
12130733,116962523,207,24-11-01,US,NaN,https://www.broadwayworld.com/article/Photos-A...,Alaska Thunderf*ck & More in DRAG: THE MUSICAL
12130734,116962524,249,24-11-01,US,NaN,https://www.broadwayworld.com/article/Photos-R...,Ryan Reynolds and Blake Lively Visit & JULIET


In [21]:
# Exclude rows with any NaN values
all_sources = all_sources.dropna()

# Display the cleaned DataFrame
all_sources

,textID,words,date,country,source,url,headline
0,103118757,307,23-11-01,US,nytimes.com,https://www.mercurynews.com/2023/10/31/google-...,Google trims Bay Area jobs at offices it had l...
1,103118769,961,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/yac...,Yacolt teen works with Nike to make sneakers t...
2,103118770,422,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cla...,"Clark County housing market heats over summer,..."
3,103118772,399,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/oct/31/van...,Vancouver-based ZoomInfo reports revenue incre...
4,103118775,212,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cli...,Climate solutions topic of Nov. 15 TEDx event ...
...,...,...,...,...,...,...,...
12529232,89566655,3298.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/11-men-share-e...,11 men share their experiences of abortion aft...
12529233,89566656,614.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/yes-ice-cream-...,"Yes, You Can Have Ice Cream Delivered Straight..."
12529234,89566657,287.0,22-06-30,US,Yahoo,https://www.yahoo.com/news/football-rumours-bo...,Borussia Dortmund set Jude Bellingham's price ...
12529235,89566570,384.0,22-06-30,US,The Hill,https://thehill.com/changing-america/enrichmen...,New Jersey is losing residents to states like ...


In [22]:
# check how many articles there are from each country
country_count = all_sources['country'].value_counts()
print(country_count)

country
US    5104690
GB    1572435
IN    1517738
CA     654027
IE     637629
AU     631702
NG     559090
ZA     332104
PK     241562
NZ     222043
PH     220959
SG     202764
KE     151783
MY     148577
??      91510
LK      69392
HK      46729
GH      45649
BD      41574
JM      16782
TZ      16517
ph       2957
us        828
UK         13
Name: count, dtype: int64


In [23]:
# filter to only include sources from the US
all_sources = all_sources[all_sources['country'] == 'US']

# check length to compare with the dropbox_text file
len(all_sources)

5104690

In [24]:
# Search for the row containing the specific text ID
specific_source_id = 93233590  # Replace with your desired textID
row = all_sources.loc[all_sources["textID"] == specific_source_id]

# Display the result
row

,textID,words,date,country,source,url,headline


In [25]:
# Convert 'textID' to numeric, and raise an error if there are invalid values
all_sources['textID'] = pd.to_numeric(all_sources['textID'], errors='raise')

# Convert 'textID' to integers
all_sources['textID'] = all_sources['textID'].astype(int)

# Verify the data type
print(all_sources.dtypes)  # 'textID' should now be int64

textID       int64
words       object
date        object
country     object
source      object
url         object
headline    object
dtype: object


/tmp/ipykernel_625/3504834423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sources['textID'] = pd.to_numeric(all_sources['textID'], errors='raise')
/tmp/ipykernel_625/3504834423.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sources['textID'] = all_sources['textID'].astype(int)


In [26]:
print(type(all_sources))

<class 'pandas.core.frame.DataFrame'>


In [27]:
# check that textID is integer
all_sources[all_sources['textID'] == 93233590]

,textID,words,date,country,source,url,headline
12145855,93233590,5839.0,22-09-01,US,nytimes.com,https://www.nytimes.com/interactive/2022/09/01...,Focus GroupThese 12 Teachers Have a Word or Tw...


In [28]:
# ensure we have the desired months included
print(all_sources[all_sources['date'] == '23-03-05'])

Empty DataFrame
Columns: [textID, words, date, country, source, url, headline]
Index: []


In [41]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/all_sources.csv"

# Save the filtered DataFrame as a CSV file
all_sources.to_csv(output_file_path, index=False, encoding='utf-8')

In [3]:
# read all_sources csv
file_path = "/work/LauraSørineVoldgaard#8191/data/all_sources.csv"
all_sources = pd.read_csv(file_path)
all_sources

,textID,words,date,country,source,url,title
0,103118757,307.0,23-11-01,US,nytimes.com,https://www.mercurynews.com/2023/10/31/google-...,Google trims Bay Area jobs at offices it had l...
1,103118769,961.0,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/yac...,Yacolt teen works with Nike to make sneakers t...
2,103118770,422.0,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cla...,"Clark County housing market heats over summer,..."
3,103118772,399.0,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/oct/31/van...,Vancouver-based ZoomInfo reports revenue incre...
4,103118775,212.0,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cli...,Climate solutions topic of Nov. 15 TEDx event ...
...,...,...,...,...,...,...,...
4813988,89566655,3298.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/11-men-share-e...,11 men share their experiences of abortion aft...
4813989,89566656,614.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/yes-ice-cream-...,"Yes, You Can Have Ice Cream Delivered Straight..."
4813990,89566657,287.0,22-06-30,US,Yahoo,https://www.yahoo.com/news/football-rumours-bo...,Borussia Dortmund set Jude Bellingham's price ...
4813991,89566570,384.0,22-06-30,US,The Hill,https://thehill.com/changing-america/enrichmen...,New Jersey is losing residents to states like ...


In [26]:
print(type(all_sources))

<class 'pandas.core.frame.DataFrame'>


### Convert text to csv

In [44]:
# checking where the mismatch between textID and body occurs

# define the parent folder path
parent_folder = "/work/LauraSørineVoldgaard#8191/data/text"

# initialize an empty list to hold dataframes for each file
all_texts = []

# walk through all subdirectories and files in the parent folder
for root, dirs, files in os.walk(parent_folder):
    for file_name in files:
        file_path = os.path.join(root, file_name)  # Full file path
        
        # check if the file ends with '.txt'
        if file_name.endswith('.txt'):
            # open the file and read its contents into a string
            with open(file_path, 'r', encoding='utf-8') as file:
                raw_text = file.read()  # read all text data from the file
            
            # preprocess the text
            sample = raw_text
            sample = re.sub(" ([.,?!':])", r"\1", sample)  # punctuation characters are all needlessly preceded by a space, remove that space
            sample = re.sub("@ @ @ @ @ @ @ @ @ @", "CENSOREDfrfrfr", sample)  # replace the keyword chosen to mark words or entities that have been censored

            # split the text into articles based on '@@' markers
            article_ids = re.findall(r"@@(\d+)", sample)  # extract list of all article IDs by matching pattern of any sequence of digits following exactly two @
            articles = re.split(r'"?@@\d+ ', sample)[1:]  # split articles on article IDs, i.e. a sequence of digits preceded by exactly two @s and sometimes a quotation mark preceding that
            articles = [art[art.find("<p> ") + 4:].strip().replace(" <p> ", "\n") for art in articles]  # for each article, skip to the first paragraph tag to exclude heading/title text, then replace subsequent markers for new paragraphs with line breaks. OBS maybe 'words' column contains incorrect n words because of this
            
            # check that we found equally many IDs and article bodies
            # if number of IDs and article bodies match
            if len(article_ids) == len(articles):
                text = pd.DataFrame(data=dict(textID=article_ids, body=articles))  # create dataframe with extracted IDs and corresponding article contents
                text["textID"] = text["textID"].astype(int)  # cast textIDs to int to enable merging with `sources` dataframe
                all_texts.append(text)  # append the dataframe to the all_texts list
            # if number of IDs and article bodies don't match
            else:
                print(f"Mismatch in IDs and articles in file: {file_path}")  # prints message stating the file with the mismatch
                
                # Debug the mismatch: Print the mismatched IDs and articles
                print(f"Number of IDs: {len(article_ids)}, Number of Articles: {len(articles)}")
                
                # Find where the mismatch occurs
                for idx, (article_id, article_body) in enumerate(zip(article_ids, articles)):
                    if article_body.strip() == "":  # Check if the article body is empty
                        print(f"Empty article body detected for textID @@{article_id}")
                        break
                else:
                    # If no empty bodies, check if there's an extra ID or body
                    if len(article_ids) > len(articles):
                        print(f"Extra textID found: @@{article_ids[len(articles)]}")
                    elif len(articles) > len(article_ids):
                        print(f"Extra article body detected: {articles[len(article_ids)]}")
                continue  # moves to the next iteration of the loop

# concatenate all dataframes from the all_texts list into one dataframe
all_texts = pd.concat(all_texts, ignore_index=True)

# display dataframe
all_texts


Mismatch in IDs and articles in file: /work/LauraSørineVoldgaard#8191/data/text/text-21-05/21-05-us4.txt
Number of IDs: 14203, Number of Articles: 14202
Empty article body detected for textID @@32795041
Mismatch in IDs and articles in file: /work/LauraSørineVoldgaard#8191/data/text/text-24-11/24-11-us3.txt
Number of IDs: 21463, Number of Articles: 21462
Empty article body detected for textID @@118192649


,textID,body
0,90885241,"In addition, First Republic sought to woo its ..."
1,90885245,President Joe Biden reiterated U.S. commitment...
2,90885251,"By DAVID RISINGMay 1, 2023 GMT\nFire and smoke..."
3,90885252,BOTTOM LINE: The Houston Astros host the San F...
4,90885254,"By ANDREW DALTONMay 1, 2023 GMT\n1 of 2\nFILE ..."
...,...,...
5002588,88367713,You're reading a free article with opinions th...
5002589,88367714,Founded in 1993 by brothers Tom and David Gard...
5002590,88367715,The office of New York Attorney General Letiti...
5002591,88367718,"December is usually a quiet month for gaming, ..."


In [45]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/all_texts.csv"

# Save the filtered DataFrame as a CSV file
all_texts.to_csv(output_file_path, index=False, encoding='utf-8')

In [29]:
# load files pre merge

# read all_texts csv
file_path = "/work/LauraSørineVoldgaard#8191/data/all_texts.csv"
all_texts = pd.read_csv(file_path)
all_texts

,textID,body
0,90885241,"In addition, First Republic sought to woo its ..."
1,90885245,President Joe Biden reiterated U.S. commitment...
2,90885251,"By DAVID RISINGMay 1, 2023 GMT\nFire and smoke..."
3,90885252,BOTTOM LINE: The Houston Astros host the San F...
4,90885254,"By ANDREW DALTONMay 1, 2023 GMT\n1 of 2\nFILE ..."
...,...,...
5002588,88367713,You're reading a free article with opinions th...
5002589,88367714,Founded in 1993 by brothers Tom and David Gard...
5002590,88367715,The office of New York Attorney General Letiti...
5002591,88367718,"December is usually a quiet month for gaming, ..."


In [30]:
print(type(all_texts))

<class 'pandas.core.frame.DataFrame'>


In [31]:
# number of duplicate rows (after removing the NaN in body column rows)
duplicates = all_texts[all_texts['textID'].duplicated(keep=False)]
duplicates

,textID,body
3300,98657050,NaN
3301,98657050,<p> BlackRock BlackRock CEO CEO Larry Larry Fi...
3302,98657051,NaN
3303,98657051,<p> A A Ukrainian Ukrainian serviceman service...
3304,98657053,NaN
...,...,...
4987107,88281308,<p> We We reported reported on on this this la...
4987108,88281309,NaN
4987109,88281309,<p> Covid Covid defined defined the the politi...
4987111,88281314,NaN


In [42]:
# Filter duplicates with non-empty "body"
cleaned_duplicates = duplicates[duplicates['body'] != ""]
cleaned_duplicates

,textID,body


In [49]:
# Remove duplicates, keeping the first occurrence based on 'textID'
cleaned_duplicates2 = duplicates.drop_duplicates(subset='textID', keep='last')

# Verify that duplicates are removed
print(cleaned_duplicates2)

           textID                                               body
3301     98657050  <p> BlackRock BlackRock CEO CEO Larry Larry Fi...
3303     98657051  <p> A A Ukrainian Ukrainian serviceman service...
3305     98657053  <p> Alok Alok Kumar Kumar,, left left,, census...
3307     98657054  <p> SEOUL SEOUL -- -- After After a a 12-year ...
3320     98657351  <p> WASHINGTON WASHINGTON -- -- The The " " si...
...           ...                                                ...
4987099  88281203  <p> Few Few things things are are more more ti...
4987103  88281214  <p> -- -- Recommendations Recommendations are ...
4987107  88281308  <p> We We reported reported on on this this la...
4987109  88281309  <p> Covid Covid defined defined the the politi...
4987112  88281314  <p> In In 1910 1910,, this this story story on...

[142506 rows x 2 columns]


In [34]:
# Display rows with NaN values in any column
nan = all_texts[all_texts.isna().any(axis=1)]

# Print the rows with NaN values
nan

,textID,body


In [32]:
# 👹
all_texts = all_texts.drop_duplicates(subset='textID', keep='last')
all_texts

,textID,body
0,90885241,"In addition, First Republic sought to woo its ..."
1,90885245,President Joe Biden reiterated U.S. commitment...
2,90885251,"By DAVID RISINGMay 1, 2023 GMT\nFire and smoke..."
3,90885252,BOTTOM LINE: The Houston Astros host the San F...
4,90885254,"By ANDREW DALTONMay 1, 2023 GMT\n1 of 2\nFILE ..."
...,...,...
5002588,88367713,You're reading a free article with opinions th...
5002589,88367714,Founded in 1993 by brothers Tom and David Gard...
5002590,88367715,The office of New York Attorney General Letiti...
5002591,88367718,"December is usually a quiet month for gaming, ..."


In [33]:
print(type(all_texts))

<class 'pandas.core.frame.DataFrame'>


In [34]:
# Search for the row containing the specific text ID
specific_text_id = 93233590  # Replace with your desired textID
row = all_texts.loc[all_texts['textID'] == specific_text_id]

# Display the result
row

,textID,body
2636503,93233590,"<p> In In one one word word,, how how does doe..."


In [35]:
# Convert 'textID' to numeric, and raise an error if there are invalid values
#sources['textID'] = pd.to_numeric(sources['textID'], errors='raise')

# Convert 'textID' to integers
all_texts['textID'] = all_texts['textID'].astype(int)

# Verify the data type
print(all_texts.dtypes)  # 'textID' should now be int64

textID     int64
body      object
dtype: object


/tmp/ipykernel_625/3619771529.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_texts['textID'] = all_texts['textID'].astype(int)


### Merge sources and texts

In [36]:
all_articles = all_sources.merge(all_texts, on = "textID", how = "inner") # join sources dataframe with their corresponding article content using the textIDs, keeping only the articles where text IDs match. How = left to ensure that it is a dataframe and series, not a list
all_articles

,textID,words,date,country,source,url,headline,body
0,103118757,307,23-11-01,US,nytimes.com,https://www.mercurynews.com/2023/10/31/google-...,Google trims Bay Area jobs at offices it had l...,Google has revealed plans to trim dozens of jo...
1,103118769,961,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/yac...,Yacolt teen works with Nike to make sneakers t...,"Macey Bodily of Yacolt, 15, holds her custom-d..."
2,103118770,422,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cla...,"Clark County housing market heats over summer,...",The median sale price for Southwest Washington...
3,103118772,399,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/oct/31/van...,Vancouver-based ZoomInfo reports revenue incre...,"ZoomInfo, formerly DiscoverOrg, is based in Va..."
4,103118775,212,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cli...,Climate solutions topic of Nov. 15 TEDx event ...,"Learn the latest and local "" ideas worth sprea..."
...,...,...,...,...,...,...,...,...
4702689,89566655,3298.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/11-men-share-e...,11 men share their experiences of abortion aft...,BY Emily Ash Powell ( with additional reportin...
4702690,89566656,614.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/yes-ice-cream-...,"Yes, You Can Have Ice Cream Delivered Straight...",McConnell's has been scooping ice cream for mo...
4702691,89566657,287.0,22-06-30,US,Yahoo,https://www.yahoo.com/news/football-rumours-bo...,Borussia Dortmund set Jude Bellingham's price ...,Jude Bellingham has become part of the England...
4702692,89566570,384.0,22-06-30,US,The Hill,https://thehill.com/changing-america/enrichmen...,New Jersey is losing residents to states like ...,A new report from the travel site moveBuddha s...


In [10]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_pre_preprocessing.csv"

# Save the filtered DataFrame as a CSV file
all_articles.to_csv(output_file_path, index=False, encoding='utf-8')

In [3]:
# read csv so you don't have to run all the above again
file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_pre_preprocessing.csv"
all_articles_pre_preprocessing = pd.read_csv(file_path)
all_articles_pre_preprocessing

KeyboardInterrupt: 

### Removing duplicate rows - step not necessary anymore

In [54]:
print(type(all_articles))  # Should output: <class 'pandas.core.frame.DataFrame'>
print(type(all_articles['body']))  # Should output: <class 'pandas.core.series.Series'>

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [37]:
# Search for the row containing the specific text ID
specific_text_id = 93233590  # Replace with your desired textID
row = all_articles.loc[all_articles['textID'] == specific_text_id]

# Display the result
row

,textID,words,date,country,source,url,headline,body
4370781,93233590,5839.0,22-09-01,US,nytimes.com,https://www.nytimes.com/interactive/2022/09/01...,Focus GroupThese 12 Teachers Have a Word or Tw...,"<p> In In one one word word,, how how does doe..."


In [56]:
# Count the number of rows where the "body" column is NaN or empty
empty_body_count = all_articles['body'].isna().sum()
print(empty_body_count)

0


In [57]:

# Remove rows where "body" is NaN or an empty string
all_articles_cleaned = all_articles[~(all_articles['body'].isna())]
all_articles_cleaned

,textID,words,date,country,source,url,headline,body
0,103118757,307,23-11-01,US,nytimes.com,https://www.mercurynews.com/2023/10/31/google-...,Google trims Bay Area jobs at offices it had l...,Google has revealed plans to trim dozens of jo...
1,103118769,961,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/yac...,Yacolt teen works with Nike to make sneakers t...,"Macey Bodily of Yacolt, 15, holds her custom-d..."
2,103118770,422,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cla...,"Clark County housing market heats over summer,...",The median sale price for Southwest Washington...
3,103118772,399,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/oct/31/van...,Vancouver-based ZoomInfo reports revenue incre...,"ZoomInfo, formerly DiscoverOrg, is based in Va..."
4,103118775,212,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cli...,Climate solutions topic of Nov. 15 TEDx event ...,"Learn the latest and local "" ideas worth sprea..."
...,...,...,...,...,...,...,...,...
4702689,89566655,3298.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/11-men-share-e...,11 men share their experiences of abortion aft...,BY Emily Ash Powell ( with additional reportin...
4702690,89566656,614.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/yes-ice-cream-...,"Yes, You Can Have Ice Cream Delivered Straight...",McConnell's has been scooping ice cream for mo...
4702691,89566657,287.0,22-06-30,US,Yahoo,https://www.yahoo.com/news/football-rumours-bo...,Borussia Dortmund set Jude Bellingham's price ...,Jude Bellingham has become part of the England...
4702692,89566570,384.0,22-06-30,US,The Hill,https://thehill.com/changing-america/enrichmen...,New Jersey is losing residents to states like ...,A new report from the travel site moveBuddha s...


In [19]:
# Efficiently remove rows with empty or whitespace-only values in the 'body' column
#articles_merged = articles_merged[articles_merged['body'].str.strip() != ""]

# Reset the index after filtering
#articles_merged.reset_index(drop=True, inplace=True)

# Check the shape of the cleaned DataFrame
#print(f"Rows after cleaning: {len(articles_merged)}")

# Check that the empty rows have efficiently been removed from articles dataframe 
#articles_merged

Rows after cleaning: 3923767


,textID,words,date,country,source,url,title,body
0,93233590,5839.0,22-09-01,US,nytimes.com,https://www.nytimes.com/interactive/2022/09/01...,Focus GroupThese 12 Teachers Have a Word or Tw...,"<p> In In one one word word,, how how does doe..."
1,93233624,1705.0,22-09-01,US,washingtonpost.com,https://www.washingtonpost.com/health/2022/09/...,BREAKING NEWS,The Centers for Disease Control and Prevention...
2,93233626,1981.0,22-09-01,US,washingtonpost.com,https://www.washingtonpost.com/science/2022/09...,"In summer of viruses, new disease outbreaks be...",In 13 years as an infectious-disease doctor in...
3,93233627,1327.0,22-09-01,US,washingtonpost.com,https://www.washingtonpost.com/investigations/...,Ginni Thomas pressed Wisconsin lawmakers to ov...,"Virginia "" Ginni "" Thomas, the conservative ac..."
4,93233628,1601.0,22-09-01,US,washingtonpost.com,https://www.washingtonpost.com/politics/2022/0...,POST POLITICS NOW,"Washington, DC - September 1: President Joe Bi..."
...,...,...,...,...,...,...,...,...
3923762,53599515,1891.0,22-01-31,US,tmcnet.com,https://www.tmcnet.com/topics/articles/2022/01...,Tools &amp; Tricks to Improve Employee Perform...,HR managers should measure employee performanc...
3923763,53599516,357.0,22-01-31,US,tmcnet.com,https://www.tmcnet.com/usubmit/-asures-payroll...,Asure's Payroll Fintech Powers New Treasury Sy...,""" Automating the ins and outs of money movemen..."
3923764,53599517,295.0,22-01-31,US,tmcnet.com,https://www.tmcnet.com/usubmit/-decibel-therap...,Decibel Therapeutics to Present at the 45th An...,Decibel Therapeutics to Present at the 45th An...
3923765,53599518,588.0,22-01-31,US,tmcnet.com,https://www.tmcnet.com/usubmit/-immixbio-initi...,ImmixBio Initiates IMX-110 GMP Manufacturing S...,""" Initiating GMP manufacturing 36 days after t..."


In [22]:
# checking that removing duplicate rows worked

# Search for the row containing the specific text ID
specific_text_id = 93233590  # Replace with your desired textID
row = all_articles_cleaned.loc[all_articles_cleaned['textID'] == specific_text_id]

# Display the result
row

,textID,words,date,country,source,url,title,body
4514235,93233590,5839.0,22-09-01,US,nytimes.com,https://www.nytimes.com/interactive/2022/09/01...,Focus GroupThese 12 Teachers Have a Word or Tw...,"<p> In In one one word word,, how how does doe..."


In [23]:
# Convert 'textID' to numeric, and raise an error if there are invalid values
#sources['textID'] = pd.to_numeric(sources['textID'], errors='raise')

# Convert 'textID' to integers
#sources['textID'] = sources['textID'].astype(int)

# Verify the data type
print(all_articles_cleaned.dtypes)  # 'textID' should now be int64

textID       int64
words      float64
date        object
country     object
source      object
url         object
title       object
body        object
dtype: object


In [25]:
# change column name 'title' to 'headline'
all_articles_cleaned = all_articles_cleaned.rename(columns={"title": "headline"})
all_articles_cleaned

,textID,words,date,country,source,url,headline,body
0,103118757,307.0,23-11-01,US,nytimes.com,https://www.mercurynews.com/2023/10/31/google-...,Google trims Bay Area jobs at offices it had l...,Google has revealed plans to trim dozens of jo...
1,103118769,961.0,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/yac...,Yacolt teen works with Nike to make sneakers t...,"Macey Bodily of Yacolt, 15, holds her custom-d..."
2,103118770,422.0,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cla...,"Clark County housing market heats over summer,...",The median sale price for Southwest Washington...
3,103118772,399.0,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/oct/31/van...,Vancouver-based ZoomInfo reports revenue incre...,"ZoomInfo, formerly DiscoverOrg, is based in Va..."
4,103118775,212.0,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cli...,Climate solutions topic of Nov. 15 TEDx event ...,"Learn the latest and local "" ideas worth sprea..."
...,...,...,...,...,...,...,...,...
4713033,89566655,3298.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/11-men-share-e...,11 men share their experiences of abortion aft...,BY Emily Ash Powell ( with additional reportin...
4713034,89566656,614.0,22-06-30,US,Yahoo,https://www.yahoo.com/lifestyle/yes-ice-cream-...,"Yes, You Can Have Ice Cream Delivered Straight...",McConnell's has been scooping ice cream for mo...
4713035,89566657,287.0,22-06-30,US,Yahoo,https://www.yahoo.com/news/football-rumours-bo...,Borussia Dortmund set Jude Bellingham's price ...,Jude Bellingham has become part of the England...
4713036,89566570,384.0,22-06-30,US,The Hill,https://thehill.com/changing-america/enrichmen...,New Jersey is losing residents to states like ...,A new report from the travel site moveBuddha s...


### Adding merged_23_03 data

In [58]:
# load in merged_23_03 data
file_path = "/work/LauraSørineVoldgaard#8191/data/merged_23_03.csv"
merged_23_03 = pd.read_csv(file_path)
merged_23_03

,textID,words,date,country,source,url,headline,body
0,90659700,827,23-03-01,US,YAHOO!News,https://news.yahoo.com/belarus-leader-fully-su...,Belarus leader 'fully supports' China's Ukrain...,Belarus President Alexander Lukashenko told hi...
1,90659707,184,23-03-01,US,YAHOO!News,https://news.yahoo.com/jalen-carter-charged-re...,Jalen Carter charged with reckless driving in ...,On Wednesday at 10:30 AM at the NFL Scouting C...
2,90659710,2344,23-03-01,US,YAHOO!News,https://news.yahoo.com/male-contraceptive-pill...,`The male contraceptive pill is on its way - b...,"Eloise Hendy\nMarch 1, 2023, 8:07 AM*9 min rea..."
3,90659714,415,23-03-01,US,New York Post,https://nypost.com/2023/02/09/ex-fbi-agent-nic...,Ex-FBI agent Nicole Parker: Bureau `politicall...,Former FBI Special Agent Nicole Parker testifi...
4,90659753,441,23-03-01,US,New York Post,https://nypost.com/2023/02/28/brendan-fraser-r...,Brendan Fraser reveals he almost died shooting...,""" I was choked out accidentally, "" said Fraser..."
...,...,...,...,...,...,...,...,...
71009,97896045,331,23-03-31,US,wvnews.com,https://wvnews.com/prestoncountynews/news/tunn...,Tunnelton man charged with making explosive de...,TUNNELTON -- A Tunnelton man was charged with ...
71010,97896046,694,23-03-31,US,wvnews.com,https://wvnews.com/sports/highschool/grafton-g...,Grafton girls track take 7th in season opening...,"BRIDGEPORT, W.Va. ( WV News ) -- The Grafton g..."
71011,97896047,805,23-03-31,US,wvnews.com,https://wvnews.com/sports/highschool/rcb-softb...,RCB softball scores 31 runs in seven innings t...,"CLARKSBURG, W.Va. ( WV News ) -- Robert C. Byr..."
71012,97896048,837,23-03-31,US,wvnews.com,https://wvnews.com/sports/highschool/rcb-withs...,RCB withstands Elkins' late comeback,"CLARKSBURG, W.Va. ( WV News ) -- A day after e..."


In [60]:
# concatenate with merged_23_03
all_articles = pd.concat([all_articles, merged_23_03], ignore_index=True)

# Display the combined DataFrame
all_articles

,textID,words,date,country,source,url,headline,body
0,103118757,307,23-11-01,US,nytimes.com,https://www.mercurynews.com/2023/10/31/google-...,Google trims Bay Area jobs at offices it had l...,Google has revealed plans to trim dozens of jo...
1,103118769,961,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/yac...,Yacolt teen works with Nike to make sneakers t...,"Macey Bodily of Yacolt, 15, holds her custom-d..."
2,103118770,422,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cla...,"Clark County housing market heats over summer,...",The median sale price for Southwest Washington...
3,103118772,399,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/oct/31/van...,Vancouver-based ZoomInfo reports revenue incre...,"ZoomInfo, formerly DiscoverOrg, is based in Va..."
4,103118775,212,23-11-01,US,startribune.com,https://www.columbian.com/news/2023/nov/01/cli...,Climate solutions topic of Nov. 15 TEDx event ...,"Learn the latest and local "" ideas worth sprea..."
...,...,...,...,...,...,...,...,...
4773703,97896045,331,23-03-31,US,wvnews.com,https://wvnews.com/prestoncountynews/news/tunn...,Tunnelton man charged with making explosive de...,TUNNELTON -- A Tunnelton man was charged with ...
4773704,97896046,694,23-03-31,US,wvnews.com,https://wvnews.com/sports/highschool/grafton-g...,Grafton girls track take 7th in season opening...,"BRIDGEPORT, W.Va. ( WV News ) -- The Grafton g..."
4773705,97896047,805,23-03-31,US,wvnews.com,https://wvnews.com/sports/highschool/rcb-softb...,RCB softball scores 31 runs in seven innings t...,"CLARKSBURG, W.Va. ( WV News ) -- Robert C. Byr..."
4773706,97896048,837,23-03-31,US,wvnews.com,https://wvnews.com/sports/highschool/rcb-withs...,RCB withstands Elkins' late comeback,"CLARKSBURG, W.Va. ( WV News ) -- A day after e..."


In [61]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles.csv"

# Save the filtered DataFrame as a CSV file
all_articles.to_csv(output_file_path, index=False, encoding='utf-8')

### Merge with AllSides dataframe

In [62]:
# read allsides_left csv
file_path = "/work/LauraSørineVoldgaard#8191/data/AllSides/allsides_left.csv"
allsides_left = pd.read_csv(file_path)
allsides_left

,allsides_media_bias_ratings.publication.source_name,allsides_media_bias_ratings.publication.source_type,allsides_media_bias_ratings.publication.media_bias_rating,allsides_media_bias_ratings.publication.source_url,allsides_media_bias_ratings.publication.allsides_url
0,AframNews,News Media,Left,https://aframnews.com/,https://www.allsides.com/news-source/aframnews...
1,Afro,News Media,Left,https://afro.com/,https://www.allsides.com/news-source/afro-medi...
2,AJ+,News Media,Left,https://www.ajplus.net/,https://www.allsides.com/news-source/aj-media-...
3,AlterNet,News Media,Left,http://www.alternet.org/,https://www.allsides.com/news-source/alternet-...
4,Aquinas College Saint,News Media,Left,https://thesaintaq.com/,https://www.allsides.com/news-source/aquinas-c...
...,...,...,...,...,...
98,Upworthy,News Media,Left,http://upworthy.com,https://www.allsides.com/news-source/upworthy
99,Vice,News Media,Left,https://www.vice.com/en,https://www.allsides.com/news-source/vice-medi...
100,Vogue,News Media,Left,https://www.vogue.com/,https://www.allsides.com/news-source/vogue-med...
101,Vox,News Media,Left,http://www.vox.com,https://www.allsides.com/news-source/vox-news-...


In [63]:
# read allsides_right csv
file_path = "/work/LauraSørineVoldgaard#8191/data/AllSides/allsides_right.csv"
allsides_right = pd.read_csv(file_path)
allsides_right

,allsides_media_bias_ratings.publication.source_name,allsides_media_bias_ratings.publication.source_type,allsides_media_bias_ratings.publication.media_bias_rating,allsides_media_bias_ratings.publication.source_url,allsides_media_bias_ratings.publication.allsides_url
0,African American Conservatives,News Media,Right,http://africanamericanconservatives.com/,https://www.allsides.com/news-source/african-a...
1,Alaska Watchman,News Media,Right,https://alaskawatchman.com/,https://www.allsides.com/news-source/alaska-wa...
2,Alpha News,News Media,Right,https://alphanews.org/,https://www.allsides.com/news-source/alpha-new...
3,American Greatness,News Media,Right,https://amgreatness.com/,https://www.allsides.com/news-source/american-...
4,American Thinker,News Media,Right,http://www.americanthinker.com/,https://www.allsides.com/news-source/american-...
...,...,...,...,...,...
77,Virtue Online,News Media,Right,https://virtueonline.org/,https://www.allsides.com/news-source/virtue-on...
78,Washington Free Beacon,News Media,Right,http://freebeacon.com/,https://www.allsides.com/news-source/washingto...
79,Whatfinger News,News Media,Right,https://www.whatfinger.com/,https://www.allsides.com/news-source/whatfinge...
80,WND,News Media,Right,http://www.wnd.com/,https://www.allsides.com/news-source/wnd-media...


In [65]:
# concatenate the two dataframes
allsides_df = pd.concat([allsides_left, allsides_right])
allsides_df

,allsides_media_bias_ratings.publication.source_name,allsides_media_bias_ratings.publication.source_type,allsides_media_bias_ratings.publication.media_bias_rating,allsides_media_bias_ratings.publication.source_url,allsides_media_bias_ratings.publication.allsides_url
0,AframNews,News Media,Left,https://aframnews.com/,https://www.allsides.com/news-source/aframnews...
1,Afro,News Media,Left,https://afro.com/,https://www.allsides.com/news-source/afro-medi...
2,AJ+,News Media,Left,https://www.ajplus.net/,https://www.allsides.com/news-source/aj-media-...
3,AlterNet,News Media,Left,http://www.alternet.org/,https://www.allsides.com/news-source/alternet-...
4,Aquinas College Saint,News Media,Left,https://thesaintaq.com/,https://www.allsides.com/news-source/aquinas-c...
...,...,...,...,...,...
77,Virtue Online,News Media,Right,https://virtueonline.org/,https://www.allsides.com/news-source/virtue-on...
78,Washington Free Beacon,News Media,Right,http://freebeacon.com/,https://www.allsides.com/news-source/washingto...
79,Whatfinger News,News Media,Right,https://www.whatfinger.com/,https://www.allsides.com/news-source/whatfinge...
80,WND,News Media,Right,http://www.wnd.com/,https://www.allsides.com/news-source/wnd-media...


In [66]:
# merge with allsides df👹

from urllib.parse import urlparse

def normalize_url(url):
    """Normalize URL by removing protocol and www."""
    parsed_url = urlparse(url)
    return parsed_url.netloc.replace('www.', '') + parsed_url.path

def get_bias_rating_fast(article_url, normalized_allsides):
    """Efficiently find the bias rating for a given article URL."""
    normalized_article_url = normalize_url(article_url)
    # Iterate through the normalized URLs and check for a match
    for base_url, bias in normalized_allsides.items():
        if normalized_article_url.startswith(base_url):
            return bias
    return None  # No match found

# Precompute normalized URLs and create a mapping
allsides_df['normalized_source_url'] = allsides_df['allsides_media_bias_ratings.publication.source_url'].apply(normalize_url)
normalized_allsides = dict(zip(
    allsides_df['normalized_source_url'],
    allsides_df['allsides_media_bias_ratings.publication.media_bias_rating']
))

# Create a new dataframe with the matches
all_articles_allsides = all_articles.copy()
all_articles_allsides['leaning'] = all_articles_allsides['url'].apply(lambda url: get_bias_rating_fast(url, normalized_allsides))
all_articles_allsides = all_articles_allsides[all_articles_allsides['leaning'].notnull()]  # Filter only matched rows

# print the resulting dataframe
all_articles_allsides


,textID,words,date,country,source,url,headline,body,leaning
763,91713230,2172,23-11-01,US,Slate,https://slate.com/technology/2023/11/gaza-hosp...,A Medical Student in Gaza Tells Us What She's ...,Gaza's health care system is in catastrophic s...,Left
765,91713243,753,23-11-01,US,The Boston Globe,https://www.bostonglobe.com/2023/10/31/metro/i...,Involuntary commitments can be a vital lifelin...,Amid revelations that an Army reservist was se...,Left
766,91713244,748,23-11-01,US,The Boston Globe,https://www.bostonglobe.com/2023/10/31/sports/...,"Favorites, sleepers, and players to watch for ...","Four divisions, 130 teams, all shooting for a ...",Left
770,91713281,1027,23-11-01,US,The Boston Globe,https://www.bostonglobe.com/2023/11/01/sports/...,"Josh McDaniels took the fall, but owner Mark D...",Raiders owner Mark Davis ( left ) gave Josh Mc...,Left
795,91713388,598,23-11-01,US,The American Spectator,https://spectator.org/the-world-has-become-ins...,The World Has Become Instantaneous,Suppose that tomorrow the Philippines were to ...,Right
...,...,...,...,...,...,...,...,...,...
4773025,97894781,610,23-03-31,US,arkansasonline.com,https://arkansasonline.com/news/2023/mar/31/le...,"Let's Eat: Wellington's, The Piano Bar, Crisp ...",Not just for college kids anymore: The Piano B...,Left
4773026,97894782,461,23-03-31,US,arkansasonline.com,https://arkansasonline.com/news/2023/mar/31/ua...,"UA finishes 3rd, ousted at NCAA regional",The No. 17 University of Arkansas gymnastics t...,Left
4773027,97894783,838,23-03-31,US,arkansasonline.com,https://arkansasonline.com/news/2023/mar/31/ta...,Target of `Tiger King' plot giving big cats to...,A lion-tiger hybrid basks in the sunshine in M...,Left
4773569,97895748,286,23-03-31,US,buzzfeed.com,https://www.buzzfeednews.com/article/davidmack...,A Court Is Allowing Andrew Tate And His Brothe...,A Romanian court on Friday granted an appeal f...,Left


In [67]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_allsides.csv"

# Save the filtered DataFrame as a CSV file
all_articles_allsides.to_csv(output_file_path, index=False, encoding='utf-8')

In [3]:
# read articles_merged_matched csv so you don't have to run everything

# read allsides_left csv
file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_allsides.csv"
all_articles_allsides = pd.read_csv(file_path)

# --------------------------------------------

In [4]:
# filter for abortion 👹

# Convert all text in the "body" column to lowercase
all_articles_allsides['body'] = all_articles_allsides['body'].str.lower()
all_articles_allsides['headline'] = all_articles_allsides['headline'].str.lower()

# Define the abortion-related search terms
search_terms_list = [
    "abortion",
    "antiabortion",
    "abortions",
    "abortion ban",
    "abortion bans",
    "abortion right",
    "abortion rights",
    "reproductive right",
    "reproductive rights",
    "reproductive health",
    "reproductive health care",
    "reproductive healthcare",
    #"pregnancy",
    #"pregnancies",
    #"pregnant",
    "termination",
    "feticide",
    "foeticide",
    "embryotomy",
    "miscarriage",
    "aborticide",
    "fetus",
    "maternal mortality",
    #"trimester",
    "pro-choice",
    "pro-life",
    "pro choice",
    "pro life",
    "prochoice",
    "prolife",
    "roe v\\. wade",
    "dobbs v\\. jackson women's health organization",
    "planned parenthood",
    "family planning",
    "pro-lifers",
    "prolifers",
    "pro lifers",
    "pro-choicers",
    "prochoicers",
    "pro choicers",
    "unborn child",
    "unborn children",
    "post-roe",
    "post-dobbs",

]

# Combine the search terms into a single regex pattern
search_terms = r"|".join(search_terms_list)


# Check for occurrences in 'body' and 'headline' with all conditions
all_articles_allsides_abortion = all_articles_allsides[
    (all_articles_allsides['body'].str.count(search_terms, flags=re.IGNORECASE) >= 2) |  # At least 2 in body
    (all_articles_allsides['headline'].str.count(search_terms, flags=re.IGNORECASE) >= 2) |  # At least 2 in title
    (
        (all_articles_allsides['body'].str.count(search_terms, flags=re.IGNORECASE) >= 1) &  # At least 1 in body
        (all_articles_allsides['headline'].str.count(search_terms, flags=re.IGNORECASE) >= 1)   # And at least 1 in title
    )
]

# Display the filtered articles
all_articles_allsides_abortion

,textID,words,date,country,source,url,headline,body,leaning
214,91719925,723.0,23-11-02,US,The Boston Globe,https://www.bostonglobe.com/2023/11/02/metro/d...,this band plans to drown out the men's march a...,"kirk israel, an activist musician and member o...",Left
270,91721731,1248.0,23-11-03,US,Fox News,https://www.foxnews.com/politics/virginias-ele...,virginia's elections a key 2024 barometer and ...,youngkin wore a similar red vest two years ago...,Right
298,103177268,1947.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/ideas/archive/2023...,here's what biden can do to change his grim po...,"whatever your theory, it should take into acco...",Left
301,103177271,1513.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/ideas/archive/2023...,don't equate anti-zionism with anti-semitism,"on october 7, the islamist militant group hama...",Left
306,103177279,2607.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/books/archive/2023...,do you have free will?,writing a review is an exercise in free will. ...,Left
...,...,...,...,...,...,...,...,...,...
296148,97824446,1244.0,23-03-28,US,bostonglobe.com,https://bostonglobe.com/2023/03/26/metro/rev-e...,"rev. elinor lockwood yeo, reproductive rights ...","rev. elinor lockwood yeo of newton, a reproduc...",Left
296258,97847400,648.0,23-03-29,US,sfchronicle.com,https://sfchronicle.com/opinion/letterstotheed...,letters: how the country can respond to nashvi...,"enough with the "" thoughts and prayers. "" thre...",Left
296276,97848056,918.0,23-03-29,US,bostonglobe.com,https://bostonglobe.com/2023/03/28/opinion/abo...,the latest antiabortion tactic: asserting the ...,"antiabortion activist trooper elwonger, 25, fe...",Left
296507,90760423,1224.0,23-03-31,US,The Daily Beast,https://www.thedailybeast.com/satan-wants-you-...,inside the horrific (contested) abuse story th...,how did one discredited biography ignite one o...,Left


In [9]:
all_articles_allsides_abortion = all_articles_allsides_abortion_nopregnant
all_articles_allsides_abortion

,textID,words,date,country,source,url,headline,body,leaning
214,91719925,723.0,23-11-02,US,The Boston Globe,https://www.bostonglobe.com/2023/11/02/metro/d...,this band plans to drown out the men's march a...,"kirk israel, an activist musician and member o...",Left
270,91721731,1248.0,23-11-03,US,Fox News,https://www.foxnews.com/politics/virginias-ele...,virginia's elections a key 2024 barometer and ...,youngkin wore a similar red vest two years ago...,Right
298,103177268,1947.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/ideas/archive/2023...,here's what biden can do to change his grim po...,"whatever your theory, it should take into acco...",Left
301,103177271,1513.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/ideas/archive/2023...,don't equate anti-zionism with anti-semitism,"on october 7, the islamist militant group hama...",Left
306,103177279,2607.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/books/archive/2023...,do you have free will?,writing a review is an exercise in free will. ...,Left
...,...,...,...,...,...,...,...,...,...
296148,97824446,1244.0,23-03-28,US,bostonglobe.com,https://bostonglobe.com/2023/03/26/metro/rev-e...,"rev. elinor lockwood yeo, reproductive rights ...","rev. elinor lockwood yeo of newton, a reproduc...",Left
296258,97847400,648.0,23-03-29,US,sfchronicle.com,https://sfchronicle.com/opinion/letterstotheed...,letters: how the country can respond to nashvi...,"enough with the "" thoughts and prayers. "" thre...",Left
296276,97848056,918.0,23-03-29,US,bostonglobe.com,https://bostonglobe.com/2023/03/28/opinion/abo...,the latest antiabortion tactic: asserting the ...,"antiabortion activist trooper elwonger, 25, fe...",Left
296507,90760423,1224.0,23-03-31,US,The Daily Beast,https://www.thedailybeast.com/satan-wants-you-...,inside the horrific (contested) abuse story th...,how did one discredited biography ignite one o...,Left


In [10]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_allsides_abortion.csv"

# Save the filtered DataFrame as a CSV file
all_articles_allsides_abortion.to_csv(output_file_path, index=False, encoding='utf-8')

In [11]:
all_articles_allsides_abortion.sample(n=5)

,textID,words,date,country,source,url,headline,body,leaning
125318,100680085,462.0,23-08-04,US,startribune.com,https://www.foxnews.com/media/christian-singer...,christian singer mocks newspaper calling worsh...,christian singer and conservative activist sea...,Right
146634,90821026,3242.0,23-04-15,US,Rolling Stone,https://www.rollingstone.com/culture/culture-s...,all eyes on bronny james at the nike hoop summit,"lebron james's son, a four-star basketball rec...",Left
180806,87856312,819.0,21-09-01,US,New York Magazine,https://nymag.com/intelligencer/2021/09/giulia...,giuliani records cameo endorsing case against ...,freedom of speech and freedom from compelled s...,Left
293424,97421964,1763.0,23-03-09,US,theatlantic.com,https://www.theatlantic.com/politics/archive/2...,the topic biden keeps dodging,president joe biden is following a strategy of...,Left
14643,94706602,1232.0,22-11-08,US,theatlantic.com,https://www.theatlantic.com/ideas/archive/2022...,the supreme court case that could break native...,every generation of americans has seen an effo...,Left


In [83]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_allsides_abortion.csv"

# Save the filtered DataFrame as a CSV file
all_articles_allsides_abortion.to_csv(output_file_path, index=False, encoding='utf-8')

In [3]:
# read all_articles_allsides_abortion csv so you don't have to run everything

# read allsides_left csv
file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_allsides_abortion.csv"
all_articles_allsides_abortion = pd.read_csv(file_path)
all_articles_allsides_abortion

,textID,words,date,country,source,url,headline,body,leaning
0,91714081,1888.0,23-11-01,US,Slate,https://slate.com/culture/2023/11/keith-lee-at...,how keith lee single-handedly forced a reckoni...,if you have been on the internet these past fe...,Left
1,91720709,881.0,23-11-02,US,The Boston Globe,https://www.bostonglobe.com/2023/11/01/opinion...,"what do you mean by `right to shelter,' govern...","at the massachusetts state house on oct. 16, g...",Left
2,91719925,723.0,23-11-02,US,The Boston Globe,https://www.bostonglobe.com/2023/11/02/metro/d...,this band plans to drown out the men's march a...,"kirk israel, an activist musician and member o...",Left
3,103147349,1579.0,23-11-02,US,startribune.com,https://www.theatlantic.com/health/archive/202...,a modest proposal to save mothers' lives,at the busy county hospital where i did my med...,Left
4,103147360,4436.0,23-11-02,US,startribune.com,https://www.theatlantic.com/magazine/archive/1...,sex and the college girl,ever since gertrude stein made her remark abou...,Left
...,...,...,...,...,...,...,...,...,...
10165,97848056,918.0,23-03-29,US,bostonglobe.com,https://bostonglobe.com/2023/03/28/opinion/abo...,the latest antiabortion tactic: asserting the ...,"antiabortion activist trooper elwonger, 25, fe...",Left
10166,90759828,1768.0,23-03-31,US,The Atlantic,https://www.theatlantic.com/books/archive/2023...,a tale of maternal ambivalence,motherhood has always been a subject ripe for ...,Left
10167,90760423,1224.0,23-03-31,US,The Daily Beast,https://www.thedailybeast.com/satan-wants-you-...,inside the horrific (contested) abuse story th...,how did one discredited biography ignite one o...,Left
10168,97893538,2247.0,23-03-31,US,theatlantic.com,https://www.theatlantic.com/politics/archive/2...,the first electoral test of trump's indictment,the most important election of 2023 may also o...,Left


### Remove duplicate rows (not NaN but exact copies of rows), token duplicates and unnecessary characters

In [12]:
# Function to remove consecutive duplicate words
def remove_duplicates(text):
    return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

# Apply the function to the 'body' and 'headline' columns
all_articles_allsides_abortion.loc[:, 'body'] = all_articles_allsides_abortion['body'].apply(remove_duplicates)
all_articles_allsides_abortion.loc[:, 'headline'] = all_articles_allsides_abortion['headline'].apply(remove_duplicates)

# Remove the header "<p>" from the "body" column
all_articles_allsides_abortion.loc[:, 'body'] = all_articles_allsides_abortion['body'].str.replace(r'<p>', '', regex=True)

# Remove the words coded as censored
all_articles_allsides_abortion.loc[:, 'body'] = all_articles_allsides_abortion['body'].str.replace("censoredfrfrfr", "", regex=False)

all_articles_allsides_abortion

,textID,words,date,country,source,url,headline,body,leaning
214,91719925,723.0,23-11-02,US,The Boston Globe,https://www.bostonglobe.com/2023/11/02/metro/d...,this band plans to drown out the men's march a...,"kirk israel, an activist musician and member o...",Left
270,91721731,1248.0,23-11-03,US,Fox News,https://www.foxnews.com/politics/virginias-ele...,virginia's elections a key 2024 barometer and ...,youngkin wore a similar red vest two years ago...,Right
298,103177268,1947.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/ideas/archive/2023...,here's what biden can do to change his grim po...,"whatever your theory, it should take into acco...",Left
301,103177271,1513.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/ideas/archive/2023...,don't equate anti-zionism with anti-semitism,"on october 7, the islamist militant group hama...",Left
306,103177279,2607.0,23-11-03,US,inquisitr.com,https://www.theatlantic.com/books/archive/2023...,do you have free will?,writing a review is an exercise in free will. ...,Left
...,...,...,...,...,...,...,...,...,...
296148,97824446,1244.0,23-03-28,US,bostonglobe.com,https://bostonglobe.com/2023/03/26/metro/rev-e...,"rev. elinor lockwood yeo, reproductive rights ...","rev. elinor lockwood yeo of newton, a reproduc...",Left
296258,97847400,648.0,23-03-29,US,sfchronicle.com,https://sfchronicle.com/opinion/letterstotheed...,letters: how the country can respond to nashvi...,"enough with the "" thoughts and prayers. "" thre...",Left
296276,97848056,918.0,23-03-29,US,bostonglobe.com,https://bostonglobe.com/2023/03/28/opinion/abo...,the latest antiabortion tactic: asserting the ...,"antiabortion activist trooper elwonger, 25, fe...",Left
296507,90760423,1224.0,23-03-31,US,The Daily Beast,https://www.thedailybeast.com/satan-wants-you-...,inside the horrific (contested) abuse story th...,how did one discredited biography ignite one o...,Left


In [13]:
# number of duplicate rows (after removing the NaN in body column rows) - should be 0
all_articles_allsides_abortion_duplicates = all_articles_allsides_abortion[all_articles_allsides_abortion['textID'].duplicated(keep=False)]
all_articles_allsides_abortion_duplicates

,textID,words,date,country,source,url,headline,body,leaning


### Save for further analysis

In [14]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_allsides_abortion.csv"

# Save the filtered DataFrame as a CSV file
all_articles_allsides_abortion.to_csv(output_file_path, index=False, encoding='utf-8')

In [3]:
# read all_articles_allsides_abortion csv so you don't have to run everything

# read allsides_left csv
file_path = "/work/LauraSørineVoldgaard#8191/data/all_articles_allsides_abortion.csv"
all_articles_allsides_abortion = pd.read_csv(file_path)
all_articles_allsides_abortion

,textID,words,date,country,source,url,headline,body,leaning
0,91714081,1888.0,23-11-01,US,Slate,https://slate.com/culture/2023/11/keith-lee-at...,how keith lee single-handedly forced a reckoni...,if you have been on the internet these past fe...,Left
1,91720709,881.0,23-11-02,US,The Boston Globe,https://www.bostonglobe.com/2023/11/01/opinion...,"what do you mean by `right to shelter,' govern...","at the massachusetts state house on oct. 16, g...",Left
2,91719925,723.0,23-11-02,US,The Boston Globe,https://www.bostonglobe.com/2023/11/02/metro/d...,this band plans to drown out the men's march a...,"kirk israel, an activist musician and member o...",Left
3,103147349,1579.0,23-11-02,US,startribune.com,https://www.theatlantic.com/health/archive/202...,a modest proposal to save mothers' lives,at the busy county hospital where i did my med...,Left
4,103147360,4436.0,23-11-02,US,startribune.com,https://www.theatlantic.com/magazine/archive/1...,sex and the college girl,ever since gertrude stein made her remark abou...,Left
...,...,...,...,...,...,...,...,...,...
10165,97848056,918.0,23-03-29,US,bostonglobe.com,https://bostonglobe.com/2023/03/28/opinion/abo...,the latest antiabortion tactic: asserting the ...,"antiabortion activist trooper elwonger, 25, fe...",Left
10166,90759828,1768.0,23-03-31,US,The Atlantic,https://www.theatlantic.com/books/archive/2023...,a tale of maternal ambivalence,motherhood has always been a subject ripe for ...,Left
10167,90760423,1224.0,23-03-31,US,The Daily Beast,https://www.thedailybeast.com/satan-wants-you-...,inside the horrific (contested) abuse story th...,how did one discredited biography ignite one o...,Left
10168,97893538,2247.0,23-03-31,US,theatlantic.com,https://www.theatlantic.com/politics/archive/2...,the first electoral test of trump's indictment,the most important election of 2023 may also o...,Left


# Inspect data

In [15]:
# get number of articles for each political leaning

from datetime import datetime
import pandas as pd

# Ensure the 'date' column is in datetime format
all_articles_allsides_abortion = all_articles_allsides_abortion.copy()  # Create a copy to avoid SettingWithCopyWarning
all_articles_allsides_abortion['date'] = pd.to_datetime(all_articles_allsides_abortion['date'], format='%y-%m-%d')

# Create a complete range of months from '20-11' (Nov 2020) to '24-11' (Nov 2024)
start_date = datetime.strptime('20-11-01', '%y-%m-%d')
end_date = datetime.strptime('24-11-01', '%y-%m-%d')
all_months = pd.date_range(start=start_date, end=end_date, freq='MS')  # 'MS' means Month Start
all_months_formatted = all_months.strftime('%y-%m')

# Create a dictionary to hold article counts for each leaning
leanings = all_articles_allsides_abortion['leaning'].unique()
article_counts_by_leaning = {}

# Count articles per month for each leaning
for leaning in leanings:
    leaning_data = all_articles_allsides_abortion[all_articles_allsides_abortion['leaning'] == leaning].copy()  # Create a copy
    leaning_data['year_month'] = leaning_data['date'].dt.strftime('%y-%m')  # Safe modification
    articles_per_month = leaning_data['year_month'].value_counts().sort_index()
    
    # Create a Series for all months with counts, fill missing months with 0
    article_counts = pd.Series(0, index=all_months_formatted)
    article_counts.update(articles_per_month)
    
    # Save the counts to the dictionary
    article_counts_by_leaning[leaning] = article_counts

# Convert results to DataFrame for display
output_df = pd.DataFrame(article_counts_by_leaning)
output_df.index = all_months_formatted
output_df.index.name = "Month"

# Add totals for each leaning
output_df['Total'] = output_df.sum(axis=1)  # Row-wise total across leanings

# Add a row for the grand total across all months and leanings
output_df.loc['Grand Total'] = output_df.sum()  # Column-wise total across all months

# Print the results
print(output_df)


             Left  Right  Total
Month                          
20-11          67     25     92
20-12          69     58    127
21-01          82     76    158
21-02          54     57    111
21-03          66     24     90
21-04          58     37     95
21-05          74     47    121
21-06         115     55    170
21-07          89     70    159
21-08          56     38     94
21-09         288    101    389
21-10         141     78    219
21-11         103     58    161
21-12         201    103    304
22-01         112     67    179
22-02          86     54    140
22-03          93     45    138
22-04          96     46    142
22-05         327    253    580
22-06         276    238    514
22-07         361    331    692
22-08         185    165    350
22-09         172    172    344
22-10         161     93    254
22-11         139     49    188
22-12          76     34    110
23-01          78     58    136
23-02          70     24     94
23-03          66     32     98
23-04   

In [20]:
# Reset the index to include 'Month' as a column
output_df_reset = output_df.reset_index()

# Save the DataFrame to a file (e.g., CSV)
output_df_reset.to_csv("articles_distribution_leaning.csv", index=False)  # Don't include the default index

In [21]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/articles_distribution_leaning.csv"

# Save the filtered DataFrame as a CSV file
output_df_reset.to_csv(output_file_path, index=False, encoding='utf-8')

In [18]:
import pandas as pd
import re

# Function to extract the domain from a URL
def extract_domain(url):
    if pd.isnull(url):
        return None
    match = re.search(r'^(?:https?://)?(?:www\.)?([^/]+)', url)
    if match:
        return match.group(1)
    return None

# Extract domains from the 'source' column
all_articles_allsides_abortion['domain'] = all_articles_allsides_abortion['url'].apply(extract_domain)

# Create a DataFrame with unique domains and their leanings
sources_included_in_analysis = all_articles_allsides_abortion[['domain', 'leaning']].drop_duplicates()

# Rename the columns as required
sources_included_in_analysis = sources_included_in_analysis.rename(columns={
    'domain': 'Sources included in analysis',
    'leaning': 'Leaning'
})

# Sort the DataFrame alphabetically by 'Sources included in analysis'
sources_included_in_analysis = sources_included_in_analysis.sort_values(by='Sources included in analysis')

# Reset index for a clean DataFrame
sources_included_in_analysis.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
sources_included_in_analysis


,Sources included in analysis,Leaning
0,arkansasonline.com,Left
1,arktimes.com,Left
2,bizpacreview.com,Right
3,boingboing.net,Left
4,bostonglobe.com,Left
5,buzzfeednews.com,Left
6,catholicnewsagency.com,Right
7,city-journal.org,Right
8,cnsnews.com,Right
9,dailycaller.com,Right


In [19]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/sources_included_in_analysis.csv"

# Save the filtered DataFrame as a CSV file
sources_included_in_analysis.to_csv(output_file_path, index=False, encoding='utf-8')

In [38]:
# Convert the 'date' column to datetime format if it isn't already
all_articles['date'] = pd.to_datetime(all_articles['date'], format='%y-%m-%d')

# Extract the year-month part from the date
all_articles['Month'] = all_articles['date'].dt.to_period('M')  # Creates a year-month period column

# Count the number of rows for each year-month
all_articles_counts_df = all_articles['Month'].value_counts().sort_index()

# Convert to a DataFrame for a cleaner overview
all_articles_counts_df = all_articles_counts_df.reset_index()
all_articles_counts_df.columns = ['Month', 'US articles (sources and texts merged)']

# Remove the leading '20' from the 'Month' column
all_articles_counts_df['Month'] = all_articles_counts_df['Month'].astype(str).str.replace('^20', '', regex=True)

# Display the overview
print(all_articles_counts_df)


    Month  US articles (sources and texts merged)
0   20-11                                  119349
1   20-12                                  143035
2   21-01                                  165571
3   21-02                                  137059
4   21-03                                  146165
5   21-04                                  136067
6   21-05                                  125928
7   21-06                                  169491
8   21-07                                  161140
9   21-08                                  149956
10  21-09                                  160173
11  21-10                                  158080
12  21-11                                  155910
13  21-12                                  162216
14  22-01                                  206387
15  22-02                                  163977
16  22-03                                  128918
17  22-04                                  101961
18  22-05                                   85300


In [52]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/monthly_counts_df.csv"

# Save the filtered DataFrame as a CSV file
monthly_counts_df.to_csv(output_file_path, index=False, encoding='utf-8')

### Checking sources and texts files separately

In [39]:
# Convert the 'date' column to datetime format
all_sources['date'] = pd.to_datetime(all_sources['date'], format='%y-%m-%d')

# Extract the year-month part (e.g., 23-11) from the 'date' column
all_sources['Month'] = all_sources['date'].dt.to_period('M')  # Period format %Y-%m

# Count the number of rows for each month
sources_counts = all_sources['Month'].value_counts().sort_index()

# Convert to a DataFrame for better visualization
sources_counts_df = sources_counts.reset_index()
sources_counts_df.columns = ['Month', 'US articles (sources)']

# Remove the leading "20" from the "Month" column
sources_counts_df['Month'] = sources_counts_df['Month'].astype(str).str.replace('^20', '', regex=True)

# Display the resulting DataFrame
print(sources_counts_df)


    Month  US articles (sources)
0   20-11                 153182
1   20-12                 194500
2   21-01                 173147
3   21-02                 142555
4   21-03                 151831
5   21-04                 141023
6   21-05                 146528
7   21-06                 169528
8   21-07                 161193
9   21-08                 150012
10  21-09                 160199
11  21-10                 158147
12  21-11                 163092
13  21-12                 162276
14  22-01                 206454
15  22-02                 169646
16  22-03                 128961
17  22-04                 101986
18  22-05                  85300
19  22-06                  80238
20  22-07                 118911
21  22-08                 119645
22  22-09                 108644
23  22-10                  87596
24  22-11                  77393
25  22-12                  77754
26  23-01                  78687
27  23-02                  69554
28  23-04                  75738
29  23-05 

/tmp/ipykernel_625/1688406713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sources['date'] = pd.to_datetime(all_sources['date'], format='%y-%m-%d')
/tmp/ipykernel_625/1688406713.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sources['Month'] = all_sources['date'].dt.to_period('M')  # Period format %Y-%m


In [40]:
import os
import pandas as pd
import re

# Define the parent folder path
parent_folder = "/work/LauraSørineVoldgaard#8191/data/text"

# Initialize an empty list to hold dataframes for each file and folder information
all_texts = []
folder_counts = {}

# Walk through all subdirectories and files in the parent folder
for root, dirs, files in os.walk(parent_folder):
    for file_name in files:
        file_path = os.path.join(root, file_name)  # Full file path
        
        # Check if the file ends with '.txt'
        if file_name.endswith('.txt'):
            # Get the folder name
            folder_name = os.path.basename(root)  # Name of the folder where the file is located
            
            # Open the file and read its contents into a string
            with open(file_path, 'r', encoding='utf-8') as file:
                raw_text = file.read()  # Read all text data from the file
            
            # Preprocess the text
            sample = raw_text
            sample = re.sub(" ([.,?!':])", r"\1", sample)  # Remove unnecessary spaces before punctuation
            sample = re.sub("@ @ @ @ @ @ @ @ @ @", "CENSOREDfrfrfr", sample)  # Replace the censored keyword

            # Split the text into articles based on '@@' markers
            article_ids = re.findall(r"@@(\d+)", sample)  # Extract article IDs
            articles = re.split(r'"?@@\d+ ', sample)[1:]  # Split articles based on IDs
            articles = [art[art.find("<p> ") + 4:].strip().replace(" <p> ", "\n") for art in articles]

            # If the number of IDs matches the number of articles
            if len(article_ids) == len(articles):
                # Create a DataFrame
                text = pd.DataFrame(data=dict(textID=article_ids, body=articles))
                text["textID"] = text["textID"].astype(int)  # Cast textIDs to int
                text["folder"] = folder_name  # Add the folder name to each row
                all_texts.append(text)  # Append to the list
                
                # Update the count for this folder
                folder_counts[folder_name] = folder_counts.get(folder_name, 0) + len(text)
            else:
                print(f"Mismatch in IDs and articles in file: {file_path}")
                continue  # Skip to the next file

# Concatenate all dataframes from the all_texts list into one dataframe
texts4 = pd.concat(all_texts, ignore_index=True)

# Convert the folder_counts dictionary into a DataFrame for better visualization
folder_counts_df = pd.DataFrame(list(folder_counts.items()), columns=["Folder", "Row Count"]).sort_values(by="Row Count", ascending=False)

# Display the folder-wise counts
print(folder_counts_df)


Mismatch in IDs and articles in file: /work/LauraSørineVoldgaard#8191/data/text/text-21-05/21-05-us4.txt
        Folder  Row Count
45  text-22-01     211830
46  text-21-06     177145
44  text-21-12     173071
23  text-21-10     170691
14  text-21-07     168476
33  text-22-02     166337
29  text-21-01     166006
5   text-21-09     162321
47  text-21-11     160853
12  text-21-08     154411
15  text-23-03     149108
16  text-21-03     148288
7   text-20-12     143596
13  text-21-02     138003
27  text-21-04     137943
11  text-22-03     136908
20  text-20-11     132331
9   text-21-05     127270
31  text-22-08     122255
30  text-22-07     119729
26  text-22-09     114876
28  text-22-04     110914
41  text-22-10      89224
38  text-22-05      88880
8   text-22-06      85029
35  text-23-01      81092
4   text-23-06      79247
25  text-22-11      79020
0   text-23-05      78256
22  text-23-04      76718
18  text-22-12      76372
1   text-24-03      73136
21  text-23-02      72079
6   text-23

In [40]:
import os
import pandas as pd
import re

# Define the parent folder path
parent_folder = "/work/LauraSørineVoldgaard#8191/data/text"

# Initialize an empty list to hold dataframes for each file and folder information
all_texts = []
folder_counts = {}

# Walk through all subdirectories and files in the parent folder
for root, dirs, files in os.walk(parent_folder):
    for file_name in files:
        file_path = os.path.join(root, file_name)  # Full file path
        
        # Check if the file ends with '.txt'
        if file_name.endswith('.txt'):
            # Get the folder name
            folder_name = os.path.basename(root)  # Name of the folder where the file is located
            
            # Open the file and read its contents into a string
            with open(file_path, 'r', encoding='utf-8') as file:
                raw_text = file.read()  # Read all text data from the file
            
            # Preprocess the text
            sample = raw_text
            sample = re.sub(" ([.,?!':])", r"\1", sample)  # Remove unnecessary spaces before punctuation
            sample = re.sub("@ @ @ @ @ @ @ @ @ @", "CENSOREDfrfrfr", sample)  # Replace the censored keyword

            # Split the text into articles based on '@@' markers
            article_ids = re.findall(r"@@(\d+)", sample)  # Extract article IDs
            articles = re.split(r'"?@@\d+ ', sample)[1:]  # Split articles based on IDs
            articles = [art[art.find("<p> ") + 4:].strip().replace(" <p> ", "\n") for art in articles]

            # If the number of IDs matches the number of articles
            if len(article_ids) == len(articles):
                # Create a DataFrame
                text = pd.DataFrame(data=dict(textID=article_ids, body=articles))
                text["textID"] = text["textID"].astype(int)  # Cast textIDs to int
                text["folder"] = folder_name  # Add the folder name to each row
                all_texts.append(text)  # Append to the list
                
                # Update the count for this folder
                folder_counts[folder_name] = folder_counts.get(folder_name, 0) + len(text)
            else:
                print(f"Mismatch in IDs and articles in file: {file_path}")
                continue  # Skip to the next file

# Concatenate all dataframes from the all_texts list into one dataframe
texts5 = pd.concat(all_texts, ignore_index=True)

# Remove duplicates by textID, keeping the last occurrence
texts5 = texts5.drop_duplicates(subset='textID', keep='last')

# Convert the folder_counts dictionary into a DataFrame for better visualization
folder_counts_df = pd.DataFrame(list(folder_counts.items()), columns=["Folder", "Row Count"]).sort_values(by="Row Count", ascending=False)

# Display the folder-wise counts
print(folder_counts_df)

# Check the resulting DataFrame
#print(texts.head())


Mismatch in IDs and articles in file: /work/LauraSørineVoldgaard#8191/data/text/text-21-05/21-05-us4.txt
Mismatch in IDs and articles in file: /work/LauraSørineVoldgaard#8191/data/text/text-24-11/24-11-us3.txt
        Folder  Row Count
46  text-22-01     211830
47  text-21-06     177145
45  text-21-12     173071
23  text-21-10     170691
14  text-21-07     168476
34  text-22-02     166337
30  text-21-01     166006
5   text-21-09     162321
48  text-21-11     160853
12  text-21-08     154411
15  text-23-03     149108
16  text-21-03     148288
7   text-20-12     143596
13  text-21-02     138003
28  text-21-04     137943
11  text-22-03     136908
20  text-20-11     132331
9   text-21-05     127270
32  text-22-08     122255
31  text-22-07     119729
27  text-22-09     114876
29  text-22-04     110914
42  text-22-10      89224
39  text-22-05      88880
24  text-24-11      85748
8   text-22-06      85029
36  text-23-01      81092
4   text-23-06      79247
26  text-22-11      79020
0   text-2

In [41]:
# Rename the column "Folder" to "Month"
folder_counts_df = folder_counts_df.rename(columns={"Folder": "Month"})

# Use pandas string operations to remove the "text-" prefix
folder_counts_df["Month"] = folder_counts_df["Month"].str.removeprefix("text-")

# Display the updated DataFrame
print(folder_counts_df)


    Month  Row Count
46  22-01     211830
47  21-06     177145
45  21-12     173071
23  21-10     170691
14  21-07     168476
34  22-02     166337
30  21-01     166006
5   21-09     162321
48  21-11     160853
12  21-08     154411
15  23-03     149108
16  21-03     148288
7   20-12     143596
13  21-02     138003
28  21-04     137943
11  22-03     136908
20  20-11     132331
9   21-05     127270
32  22-08     122255
31  22-07     119729
27  22-09     114876
29  22-04     110914
42  22-10      89224
39  22-05      88880
24  24-11      85748
8   22-06      85029
36  23-01      81092
4   23-06      79247
26  22-11      79020
0   23-05      78256
22  23-04      76718
18  22-12      76372
1   24-03      73136
21  23-02      72079
6   23-10      66762
35  24-02      66592
3   23-07      65693
33  23-08      64990
25  24-04      62475
37  24-01      62239
2   23-09      61908
40  24-06      61907
17  24-05      59595
19  23-12      59362
38  23-11      55186
43  24-07      54953
44  24-09    

In [44]:
# Convert the "Month" column to a proper date format for sorting
folder_counts_df["Month"] = pd.to_datetime(folder_counts_df["Month"], format="%y-%m")

# Sort the DataFrame by the "Month" column
folder_counts_df = folder_counts_df.sort_values(by="Month")

# Optionally, format the "Month" column back to year-month string format
folder_counts_df["Month"] = folder_counts_df["Month"].dt.strftime("%y-%m")

# Rename the column "Row Count" to "US articles (texts)"
folder_counts_df = folder_counts_df.rename(columns={"Row Count": "US articles (texts)"})

# Reset the index for a clean display
folder_counts_df.reset_index(drop=True, inplace=True)

# Display the ordered DataFrame
print(folder_counts_df)


    Month  US articles (texts)
0   20-11               132331
1   20-12               143596
2   21-01               166006
3   21-02               138003
4   21-03               148288
5   21-04               137943
6   21-05               127270
7   21-06               177145
8   21-07               168476
9   21-08               154411
10  21-09               162321
11  21-10               170691
12  21-11               160853
13  21-12               173071
14  22-01               211830
15  22-02               166337
16  22-03               136908
17  22-04               110914
18  22-05                88880
19  22-06                85029
20  22-07               119729
21  22-08               122255
22  22-09               114876
23  22-10                89224
24  22-11                79020
25  22-12                76372
26  23-01                81092
27  23-02                72079
28  23-03               149108
29  23-04                76718
30  23-05                78256
31  23-0

In [45]:
# Merge the DataFrames on the "Month" column
merged_counts_df = all_articles_counts_df.merge(
    sources_counts_df, on="Month", how="outer", suffixes=("_articles", "_sources")
).merge(
    folder_counts_df, on="Month", how="outer"
)

# Rename the columns for clarity (optional)
merged_counts_df.rename(columns={"Row Count": "Folder Count"}, inplace=True)

# Fill NaN values with 0 (if needed, for counts)
merged_counts_df = merged_counts_df.fillna(0)

# Display the merged DataFrame
print(merged_counts_df)


    Month  US articles (sources and texts merged)  US articles (sources)  \
0   20-11                                119349.0               153182.0   
1   20-12                                143035.0               194500.0   
2   21-01                                165571.0               173147.0   
3   21-02                                137059.0               142555.0   
4   21-03                                146165.0               151831.0   
5   21-04                                136067.0               141023.0   
6   21-05                                125928.0               146528.0   
7   21-06                                169491.0               169528.0   
8   21-07                                161140.0               161193.0   
9   21-08                                149956.0               150012.0   
10  21-09                                160173.0               160199.0   
11  21-10                                158080.0               158147.0   
12  21-11   

In [47]:
# Define the file path where you want to save the CSV
output_file_path = "/work/LauraSørineVoldgaard#8191/data/scraped_articles.csv"

# Save the filtered DataFrame as a CSV file
merged_counts_df.to_csv(output_file_path, index=False, encoding='utf-8')

### Okay next steps: 1) find out why 23-03 allows so many articles to be merged together... 2) text Rebekah with your findings.

## 1) There are duplicate rows...

In [115]:
# number of duplicate rows (before removing the NaN in body column rows)
print(all_texts['textID'].duplicated().sum())    # Number of duplicate rows

# number of duplicate rows (after removing the NaN in body column rows)
duplicates = all_texts[all_texts['textID'].duplicated(keep=False)]
duplicates


TypeError: list indices must be integers or slices, not str

In [ ]:
# Remove duplicates based on the 'headline' column
all_articles_cleaned = all_articles_cleaned.drop_duplicates(subset='textID', keep='first')

# Verify if duplicates are removed
print(all_articles_cleaned['headline'].duplicated().sum())  # Should print 0
